In [ ]:
from pyspark.sql import SparkSession
data = spark.read.csv("/FileStore/tables/sharemarket.csv",schema='MARKET string, SERIES string, SYMBOL string, SECURITY string, PREV_CL_PR float, OPEN_PRICE float, HIGH_PRICE float,LOW_PRICE float, CLOSE_PRICE float, NET_TRDVAL float,NET_TRDQTY float,CORP_IND int,TRADES int,HI_52_WK float,LO_52_WK float')
data.printSchema()
data.show(truncate=False)

root
 |-- MARKET: string (nullable = true)
 |-- SERIES: string (nullable = true)
 |-- SYMBOL: string (nullable = true)
 |-- SECURITY: string (nullable = true)
 |-- PREV_CL_PR: float (nullable = true)
 |-- OPEN_PRICE: float (nullable = true)
 |-- HIGH_PRICE: float (nullable = true)
 |-- LOW_PRICE: float (nullable = true)
 |-- CLOSE_PRICE: float (nullable = true)
 |-- NET_TRDVAL: float (nullable = true)
 |-- NET_TRDQTY: float (nullable = true)
 |-- CORP_IND: integer (nullable = true)
 |-- TRADES: integer (nullable = true)
 |-- HI_52_WK: float (nullable = true)
 |-- LO_52_WK: float (nullable = true)

+------+------+----------+-------------------------+----------+----------+----------+---------+-----------+----------+----------+--------+------+--------+--------+
|MARKET|SERIES|SYMBOL    |SECURITY                 |PREV_CL_PR|OPEN_PRICE|HIGH_PRICE|LOW_PRICE|CLOSE_PRICE|NET_TRDVAL|NET_TRDQTY|CORP_IND|TRADES|HI_52_WK|LO_52_WK|
+------+------+----------+-------------------------+----------+----

In [ ]:
data = data.drop('CORP_IND')
data.show(truncate=False)

+------+------+----------+-------------------------+----------+----------+----------+---------+-----------+----------+----------+------+--------+--------+
|MARKET|SERIES|SYMBOL    |SECURITY                 |PREV_CL_PR|OPEN_PRICE|HIGH_PRICE|LOW_PRICE|CLOSE_PRICE|NET_TRDVAL|NET_TRDQTY|TRADES|HI_52_WK|LO_52_WK|
+------+------+----------+-------------------------+----------+----------+----------+---------+-----------+----------+----------+------+--------+--------+
|N     |N1    |IRFC      |BOND 8.00% PA TAX FREE S1|1086.0    |1085.0    |1085.0    |1084.54  |1085.0     |3371959.5 |3108.0    |8     |1194.0  |1050.0  |
|N     |N1    |JNPT      |BOND 6.82% PA TAX FREE S1|1001.0    |1007.1    |1015.0    |1007.1   |1015.0     |51370.0   |51.0      |2     |1529.99 |1000.0  |
|N     |N1    |NHAI      |BOND 8.20% PA TAX FREE S1|1091.38   |1092.0    |1092.0    |1087.0   |1089.98    |3861145.5 |3544.0    |39    |1140.0  |1045.0  |
|N     |N1    |NTPC      |8.41%S-R-NCD SERIES 1A   |1198.99   |1135.0 

In [ ]:
data.createOrReplaceTempView('sharemarket')

In [ ]:
#1) Display number of series present in the data
spark.sql('select count(distinct SERIES) from sharemarket').show()

+----------------------+
|count(DISTINCT SERIES)|
+----------------------+
|                    53|
+----------------------+



In [ ]:
#2) Display series present in the data
spark.sql('select distinct SERIES from sharemarket').show()

+------+
|SERIES|
+------+
|    NL|
|    NK|
|    NJ|
|    NA|
|    N3|
|    NH|
|    NE|
|    NC|
|    ND|
|    N7|
|    N8|
|    N2|
|    N5|
|    N4|
|    N6|
|    NI|
|    N1|
|    NG|
|    N9|
|    NB|
+------+
only showing top 20 rows



In [ ]:
#3) find the sum of all the prices in each series
#spark.sql('select SERIES,sum(PREV_CL_PR),sum(OPEN_PRICE),sum(HIGH_PRICE),sum(LOW_PRICE),sum(CLOSE_PRICE) from sharemarket group by SERIES').show()
spark.sql('select SERIES,sum(PREV_CL_PR+OPEN_PRICE+HIGH_PRICE+LOW_PRICE+CLOSE_PRICE) as total from sharemarket group by SERIES').show()

+------+------------------+
|SERIES|             total|
+------+------------------+
|    YH|  5119.02001953125|
|    NS|  5154.60009765625|
|    NL| 17007.05029296875|
|    NK|  9671.98974609375|
|    Z4|  5144.85009765625|
|    NJ|  21451.9697265625|
|    NX|  1625.47998046875|
|    YS|            5670.0|
|    NW|   5444.2001953125|
|    NA|  99490.4482421875|
|    Z8|            5191.0|
|    YA|  1369.97998046875|
|    NP|           6465.75|
|    N3|10440.390243530273|
|    NH| 23478.92041015625|
|    NE|  84056.6083984375|
|    NC| 71805.37841796875|
|    NQ|1560.0999755859375|
|    Z1|  6376.10986328125|
|    NR| 6161.440185546875|
+------+------------------+
only showing top 20 rows



In [ ]:
#4) Display security,series with highest net trade value
spark.sql("select SECURITY,SERIES from sharemarket where NET_TRDVAL >=(select max(NET_TRDVAL) from sharemarket)").show(truncate=False)

+-------------------------+------+
|SECURITY                 |SERIES|
+-------------------------+------+
|8.75% TAX FREE TR I S IIB|N6    |
+-------------------------+------+



In [ ]:
#5) Display series whose sum of all prices greater than net trade value
spark.sql("SELECT SERIES, (OPEN_PRICE+HIGH_PRICE+LOW_PRICE+CLOSE_PRICE) TOTAL_PRICE ,(NET_TRDVAL) TOTAL_TRADE FROM SALES WHERE OPEN_PRICE+HIGH_PRICE+LOW_PRICE+CLOSE_PRICE>NET_TRDVAL AND NET_TRDVAL IS NOT NULL").show()

In [ ]:
#6) Display series with highest net trade quantity
spark.sql('select SERIES from sharemarket where NET_TRDQTY>=(select max(NET_TRDQTY) from sharemarket)').show()

+------+
|SERIES|
+------+
|    N3|
+------+



In [ ]:
#7 Display highest and lowest open price
spark.sql('select max(OPEN_PRICE),min(OPEN_PRICE) from sharemarket').show()

+---------------+---------------+
|max(OPEN_PRICE)|min(OPEN_PRICE)|
+---------------+---------------+
|        11450.0|          13.75|
+---------------+---------------+



In [ ]:
#8 Display series which have trades more than 80
spark.sql('select series from sharemarket where TRADES>80').show()

+------+
|series|
+------+
|    N2|
|    N3|
|    N7|
+------+



In [ ]:
#Display the difference between net trade value and net trade quantity for each series
spark.sql('select SERIES, sum(NET_TRDVAL)-sum(NET_TRDQTY) as DIFF_VAL_QNT from sharemarket group by SERIES').show()

+------+------------------+
|SERIES|      DIFF_VAL_QNT|
+------+------------------+
|    YH|     61270.1015625|
|    NS|      354013.90625|
|    NL|  1581525.31640625|
|    NK|      1010121.6875|
|    Z4|     36956.6015625|
|    NJ|         7421503.0|
|    NX|  12728.7197265625|
|    YS|           79055.0|
|    NW|          203456.5|
|    NA|       3331953.875|
|    Z8|          332530.0|
|    YA|    2354.919921875|
|    NP|   316500.19921875|
|    N3|       5894945.875|
|    NH|       1686244.625|
|    NE|     2538813.90625|
|    NC|    2505038.828125|
|    NQ|            6280.0|
|    Z1|       202640.3125|
|    NR|461379.49377441406|
+------+------------------+
only showing top 20 rows

